<a href="https://colab.research.google.com/github/armandossrecife/mysentimentanalysis/blob/main/my_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 6.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

In [3]:
!pip install accelerate -U

In [4]:
from transformers import DistilBertForSequenceClassification, AutoTokenizer
from datasets import load_dataset
from transformers import TrainingArguments, Trainer  # Import from transformers
import torch

In [5]:
# Define model name and task
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
task = "sentiment-analysis"

In [6]:
# Load pre-trained DistilBERT model and tokenizer
model = DistilBertForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [7]:
# Load the IMDB movie review dataset
train_data = load_dataset("imdb", split="train")
test_data = load_dataset("imdb", split="test")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [8]:
# Function to preprocess text data
def preprocess_function(examples):
  return tokenizer(examples["text"], padding="max_length", truncation=True)

In [9]:
# Preprocess train and test data
train_data = train_data.map(preprocess_function, batched=True)
test_data = test_data.map(preprocess_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [10]:
!mkdir results

In [11]:
!ls -l

total 8
drwxr-xr-x 2 root root 4096 Jul  6 13:55 results
drwxr-xr-x 1 root root 4096 Jul  3 13:21 sample_data


In [12]:
training_args = TrainingArguments(
    output_dir="results",  # Fixed typo (removed extra space)
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=16,  # Assuming you meant "size" here
    learning_rate=2e-5,
)

In [13]:
# Create trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    compute_metrics="accuracy",
)

In [14]:
# Train the model
trainer.train()

Step,Training Loss
500,0.262100
1000,0.230400
1500,0.209900
2000,0.151200
2500,0.136600
3000,0.141700
3500,0.086800
4000,0.076800
4500,0.087300


TrainOutput(global_step=4689, training_loss=0.15044099503067873, metrics={'train_runtime': 3656.4832, 'train_samples_per_second': 20.512, 'train_steps_per_second': 1.282, 'total_flos': 9935054899200000.0, 'train_loss': 0.15044099503067873, 'epoch': 3.0})

In [25]:
new_review = "This movie was amazing! Highly recommend."
inputs = tokenizer(new_review, padding="max_length", truncation=True, return_tensors="pt")

# Move the model and input to GPU if available
if torch.cuda.is_available():
  model.to('cuda')
  inputs.to('cuda')

with torch.no_grad():
  outputs = model(**inputs)
  predictions = torch.argmax(outputs.logits, dim=-1)

# Print the predicted sentiment
if predictions == 1:
  print("Sentiment: Positive")
else:
  print("Sentiment: Negative")

Sentiment: Positive


In [26]:
my_dataset = load_dataset("imdb", split="train")
type(my_dataset)

datasets.arrow_dataset.Dataset

In [31]:
len(my_dataset['text'])

25000

In [36]:
for i, text in enumerate(my_dataset['text']):
  if i <= 5:
    print(i, text)
    print('---')

0 I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, e

In [38]:
my_sample_reviews = [
  "The animation in this superhero movie was mind-blowing! The fight scenes were so dynamic and kept me on the edge of my seat. However, the plot felt a bit predictable at times. Overall, a fun and visually stunning film. 7/10",
  "This documentary had me glued to the screen. It explored a fascinating topic with such depth and nuance. The interviews were insightful, and the visuals were beautifully crafted. A must-watch for anyone interested in [insert documentary topic]. 9/10",
  "This rom-com was a delightful escape. The leads had fantastic chemistry, and the humor landed perfectly. It wasn't groundbreaking, but it was heartwarming and entertaining. Perfect for a cozy night in. 8/10",
  "This historical drama was a disappointment. The acting was solid, but the script felt clunky and melodramatic. The historical accuracy was also questionable. It had potential, but the execution fell short. 4/10",
  "This independent film surprised me. It tackled a difficult subject matter with rawness and honesty. The performances were powerful, and the story stayed with me long after the credits rolled. A thought-provoking and impactful film. 9/10",
  "This action flick was pure adrenaline. The non-stop action sequences were thrilling, but the characters were a bit one-dimensional. Still, a great popcorn movie for a fun escape. 6/10",
  "This foreign film was a beautiful cinematic experience. The visuals were breathtaking, and the story unfolded slowly and poetically. It wasn't for everyone, but for those who appreciate slow cinema, it's a masterpiece. 8/10",
  "This horror movie did a great job building suspense. The atmosphere was creepy, and the jump scares were well-placed. However, the ending felt rushed and unsatisfying. A decent scare, but lacking in depth. 6/10",
  "This animated film was a delight for all ages. The animation style was charming, and the story was both funny and heartwarming. The characters were relatable, and there were positive messages about friendship and courage. A perfect family movie. 9/10",
  "This biopic was well-researched and informative. It captured the essence of the person's life and achievements. However, the pacing felt slow at times, and the dramatization of certain events felt unnecessary. A solid biopic, but could have been tighter. 7/10"
]

In [47]:
def analyse_review(new_review):
  inputs = tokenizer(new_review, padding="max_length", truncation=True, return_tensors="pt")

  # Move the model and input to GPU if available
  if torch.cuda.is_available():
    model.to('cuda')
    inputs.to('cuda')

  with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)

  # Print the predicted sentiment
  if predictions == 1:
    return {'Sentiment': 'Positive'}
  else:
    return {'Sentiment': 'Negative'}

In [48]:
analyse_review(new_review="This movie was amazing! Highly recommend.")

{'Sentiment': 'Positive'}

In [50]:
for review in my_sample_reviews:
  print(review)
  print("-"*50)

The animation in this superhero movie was mind-blowing! The fight scenes were so dynamic and kept me on the edge of my seat. However, the plot felt a bit predictable at times. Overall, a fun and visually stunning film. 7/10
--------------------------------------------------
This documentary had me glued to the screen. It explored a fascinating topic with such depth and nuance. The interviews were insightful, and the visuals were beautifully crafted. A must-watch for anyone interested in [insert documentary topic]. 9/10
--------------------------------------------------
This rom-com was a delightful escape. The leads had fantastic chemistry, and the humor landed perfectly. It wasn't groundbreaking, but it was heartwarming and entertaining. Perfect for a cozy night in. 8/10
--------------------------------------------------
This historical drama was a disappointment. The acting was solid, but the script felt clunky and melodramatic. The historical accuracy was also questionable. It had p

In [49]:
for review in my_sample_reviews:
  print(review)
  result = analyse_review(new_review=review)
  print(result['Sentiment'])
  print("-"*50)

The animation in this superhero movie was mind-blowing! The fight scenes were so dynamic and kept me on the edge of my seat. However, the plot felt a bit predictable at times. Overall, a fun and visually stunning film. 7/10
Positive
--------------------------------------------------
This documentary had me glued to the screen. It explored a fascinating topic with such depth and nuance. The interviews were insightful, and the visuals were beautifully crafted. A must-watch for anyone interested in [insert documentary topic]. 9/10
Positive
--------------------------------------------------
This rom-com was a delightful escape. The leads had fantastic chemistry, and the humor landed perfectly. It wasn't groundbreaking, but it was heartwarming and entertaining. Perfect for a cozy night in. 8/10
Positive
--------------------------------------------------
This historical drama was a disappointment. The acting was solid, but the script felt clunky and melodramatic. The historical accuracy was 